In [1]:
import pyOSC3
import time
import random
import copy
from collections import namedtuple, defaultdict
from subprocess import Popen
import numpy as np
import math
import json

from chord_utils import *
from dale_kojas_chords import *

In [2]:
client = pyOSC3.OSCClient()
client.connect(('127.0.0.1', 57120))

In [3]:
chords = get_chords()

# [1972, 2172, 2341, 2499, 2680, 2850, 2993, 3110]
# approved
#                ?y           y           y     y
redux_cents = [1972, 2172, 2341, 2499, 2680, 2850, 3010, 3110] +\
[1773, 1877, 1933, 2088, 2114, 2155, 2417, 2510, 2564, 2636]
#        ?y           y

# check_chords(chords, redux_cents)


In [23]:
# proc = Popen(['afplay', '/Users/a1111/Downloads/GVM097_KristeAghsdga_MVCM.mp3'])

def send_msg(note, cur_timbre, out_bus_idx):
    try:
        msg = pyOSC3.OSCMessage()
        msg.setAddress("/synths")

        msg.append([note, cur_timbre, out_bus_idx])
        # print('sent', note,'to', out_bus_idx)
        client.send(msg)
#         time.sleep(0.001)
    except Exception as e:
        pass

ref_hz = 110

prev_time = 0
sleep_time = chords[0].timestamp
# time.sleep(sleep_time)

for idx, chord in enumerate(chords):
    if idx + 1 < len(chords):
        sleep_time = chords[idx + 1].timestamp - chords[idx].timestamp
    else:
        sleep_time = 2
    tony_no_nan = []
    redux_no_nan = []
    gvm_no_nan = []
    for j, p in enumerate(chord.tony):
        if p is not None:
            tony_no_nan.append(p)
            redux_no_nan.append(chord.redux[j])
            gvm_no_nan.append(chord.gvm[j])
        break

    warning = ''
    if len(tony_no_nan) > 0:
        diff = np.max(np.abs(np.array(tony_no_nan) - redux_no_nan))
        diff = np.max(np.abs(np.array(tony_no_nan) - gvm_no_nan))
        if diff > 25:
            warning = '!!!'
    print(idx, chord.syllable, chord.gvm, chord.tony, chord.redux, warning)
    continue
    gvm_hzs = [calc_hz(pitch, ref_hz) for pitch in chord.gvm]
    tony_hzs = [calc_hz(pitch, ref_hz) for pitch in chord.tony]
    redux_hzs = [calc_hz(pitch, ref_hz) for pitch in chord.redux]
#     for hz in redux_hzs:
#     for hz in [tony_hzs[0]]:
    for hz in gvm_hzs:
        send_msg(hz, 0, 0)
    time.sleep(sleep_time)
# proc.terminate()


0 a [None, 2771, None] [None, 2771, None] [None, 2850, None] 
1 i [None, 2785, None] [None, 2787, None] [None, 2850, None] 
2 ha [None, 2798, None] [None, 2799, None] [None, 2850, None] 
3 ri [None, 2970, None] [None, 2968, None] [None, 3010, None] 
4 ro [None, 2820, None] [None, 2819, None] [None, 2850, None] 
5 ra [2280, 2657, 2863] [2274, 2664, 2856] [2341, 2680, 2850] 
6 sha [2112, 2465, 2794] [2107, 2664, 2856] [2114, 2499, 2850] 
7 re [1877, 2642, 2645] [1885, 2664, 2856] [1877, 2680, 2680] 
8 ro [1902, 2432, 2791] [1908, 2664, 2856] [1972, 2499, 2850] 
9 sha [1905, 2283, 2623] [1902, 2664, 2856] [1972, 2341, 2680] 
10 ra [2088, 2564, 2988] [2089, 2664, 2856] [2088, 2564, 3010] 
11 sha [2114, 2498, 2840] [2121, 2664, 2856] [2114, 2499, 2850] 
12 da [2253, 2802, 3097] [2314, 2664, 2856] [2341, 2850, 3110] !!!
13 le [2272, 2807, 3127] [2314, 2664, 2856] [2341, 2850, 3110] !!!
14 ko [2318, 2784, 3012] [2320, 2664, 2856] [2341, 2850, 3010] 
15 jas [2307, 2761, 2855] [2314, 2664, 2856

In [145]:
def normalize(r):
    n = [r[0], r[1]]
    while 2 <= n[0] / n[1]:
        n[1] *= 2
    while n[0] / n[1] <= 1:
        n[0] *= 2
    return tuple(n)

# wilson41
# crystal growth algorithm
primes = [3, 5, 7, 11, 13]
ratios = [(1, 1), (2, 1)]
for i, p in enumerate(primes):
    if p == 13:
        break
    r1 = [p, 1]
    max_deg = len(primes) - i
    if p == 3:
        max_deg -= 1
    for j in range(max_deg):
        ratios.append(normalize(tuple(r1)))
        r1i = [r1[1], r1[0]]
        ratios.append(normalize(tuple(r1i)))
        r1[0] *= 3
    if p == 3:
        continue
    r1 = [p, 3]
    for j in range(1, max_deg):
        ratios.append(normalize(tuple(r1)))
        r1i = [r1[1], r1[0]]
        ratios.append(normalize(tuple(r1i)))
        r1[1] *= 3

odds = [1, 3, 5, 7, 9, 11]
for p1 in odds:
    for p2 in odds:
        if p1 == p2:
            continue
        if min(p1, p2) != 1 and (max(p1, p2) % min(p1, p2) == 0):
            continue
        # wilson example
        # if (p1, p2) in [(11, 5), (5, 11)]:
            # continue
        r = (p1, p2)
        r = normalize(r)
        if r not in ratios:
            ratios.append(r)

# ratios.sort(key=lambda r: util.ratio_to_cents(r[0], r[1]))
print('ratios count', len(ratios))


ratios count 48


In [146]:
cents_tolerance = 100 / 8

ref_hz = 55

main_pitch_in_cents = 2024
main_pitch_in_hz = calc_hz(main_pitch_in_cents, ref_hz)

ret_sort = sorted(redux_cents)
ret_sort = np.array(ret_sort) - main_pitch_in_cents
print(ret_sort)

intervals = [defaultdict(int), defaultdict(int), defaultdict(int)]
intervals_as_pairs = [dict(), dict(), dict()]
intervals_as_ji = dict()

all_intervals = defaultdict(int)
all_intervals_as_pairs = dict()

cents_count = [defaultdict(int), defaultdict(int), defaultdict(int)]
all_cents_count = defaultdict(int)

all_chords = defaultdict(int)
for chord in chords:
    pitches = chord.redux
    pitches_for_diff = []
    pitches_in_hz = []
    pitches_in_cents = []
    pitches_in_cents_with_none = []
    for p_idx, pitch in enumerate(pitches):
        if pitch is None:
            pitches_in_cents_with_none.append(None)
            continue
        cents_count[p_idx][pitch] += 1
        all_cents_count[pitch] += 1
        pitches_for_diff.append(pitch)
        pitch_in_hz = calc_hz(pitch, main_pitch_in_hz)
        pitches_in_hz.append(pitch_in_hz / main_pitch_in_hz)
        pitch_in_cents = int(ratio_to_cents(pitch_in_hz, main_pitch_in_hz))
        pitches_in_cents.append(pitch_in_cents)
        pitches_in_cents_with_none.append(pitch_in_cents)

    # chord intervals
    if len(pitches_for_diff) == 1:
        diffs = []
    elif len(pitches_for_diff) == 2:
        diffs = [pitches_for_diff[1] - pitches_for_diff[0]]
    else:
        diffs = [pitches_for_diff[2] - pitches_for_diff[0],
                 pitches_for_diff[2] - pitches_for_diff[1],
                 pitches_for_diff[1] - pitches_for_diff[0]]
        pairs = [(pitches_for_diff[0], pitches_for_diff[2]),
                 (pitches_for_diff[1], pitches_for_diff[2]),
                 (pitches_for_diff[0], pitches_for_diff[1])]

        for k in range(3):
            intervals[k][diffs[k]] += 1
            all_intervals[diffs[k]] += 1
            if diffs[k] not in intervals_as_pairs[k]:
                intervals_as_pairs[k][diffs[k]] = set()
            if diffs[k] not in all_intervals_as_pairs:
                all_intervals_as_pairs[diffs[k]] = set()
            intervals_as_pairs[k][diffs[k]].add(pairs[k])
            all_intervals_as_pairs[diffs[k]].add(pairs[k])
        ch = tuple(diffs)
        all_chords[ch] += 1

    diffs = [abs(c) for c in diffs]

    notes = []
    for i, p in enumerate(diffs):
        n = str(diffs[i]) + 'c'
        state = ''
        for r in ratios:
            c1 = diffs[i]
            c2 = ratio_to_cents(r[0], r[1])
            check1 = abs(c1 - c2) < cents_tolerance
            check2 = abs(c1 - c2) < cents_tolerance * 1.5
            add = ''
            if check2 and not check1:
                add = '?'
            if check2:
                if r == (1, 1):
                    state = '.' + add
                else:
                    state = 'ji'
                    n += ' ' + add + str(r[0]) + '/' + str(r[1])
        notes.append((n, state))
#     print(chord.syllable, pitches, notes)

    if len(diffs) == 3:
        for k in range(3):
            pretty = ' '.join(notes[k][0].split(' ')[1:])
            intervals_as_ji[diffs[k]] = pretty + ' ' + notes[k][1]

for i in range(3):
    print(i)
    for k in sorted(intervals[i].keys()):
        print(k, intervals[i][k], intervals_as_pairs[i][k], intervals_as_ji[abs(k)])
    print()

print('all intervals')
for k in sorted(all_intervals.keys()):
    print(k, all_intervals[k], all_intervals_as_pairs[k], intervals_as_ji[abs(k)])
print()

    
print('cents counts')
for i in range(3):
    print(i)
    for k in sorted(cents_count[i].keys()):
        print(k, cents_count[i][k])
    print()

print('all cents')
for k in sorted(all_cents_count.keys()):
    print(k, all_cents_count[k])

print('all chords')
for ch in all_chords:
    print(all_chords[ch], ch)

[-251 -147  -91  -52   64   90  131  148  317  393  475  486  540  612
  656  826  986 1086]
0
0 1 {(2341, 2341)}  .
509 2 {(2341, 2850)} 4/3 27/20 ji
669 3 {(2341, 3010)} 40/27 ji
678 1 {(2172, 2850)} 40/27 ji
695 2 {(2155, 2850)} 3/2 ?40/27 ji
703 2 {(1933, 2636)} 3/2 ji
708 1 {(1972, 2680)} 3/2 ji
736 2 {(2114, 2850)} 32/21 ji
769 3 {(2341, 3110)} 14/9 ?11/7 ji
803 1 {(1877, 2680)} 128/81 8/5 ji
878 1 {(1972, 2850)} 5/3 ji
917 1 {(1933, 2850)} 27/16 ?12/7 ji
922 1 {(2088, 3010)} ?27/16 12/7 ji

1
0 6 {(2636, 2636), (2680, 2680), (3010, 3010), (2850, 2850), (2341, 2341)}  .
160 1 {(2850, 3010)} 12/11 11/10 ji
170 1 {(2680, 2850)} 10/9 11/10 ji
260 3 {(2850, 3110)} 7/6 ji
295 1 {(2341, 2636)} 32/27 ji
330 1 {(2680, 3010)} ?6/5 ?11/9 ji
339 1 {(2341, 2680)} 11/9 ji
340 1 {(2510, 2850)} 11/9 ji
351 4 {(2499, 2850)} 11/9 ji
433 1 {(2417, 2850)} 9/7 ?14/11 ji
446 1 {(2564, 3010)} 9/7 ji

2
0 1 {(2341, 2341)}  .
327 1 {(2172, 2499)} 6/5 ji
339 2 {(2341, 2680)} 11/9 ji
355 1 {(2155, 2510)} 

In [55]:
print(sorted(redux_cents))

[1512, 1667, 1857, 2057, 2087, 2247, 2347, 2387, 2429, 2547, 2577, 2687, 2747, 2777, 2887]
